In [4]:
import dash
from dash import dcc, html, Input, Output, ctx
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.graph_objects as go
from obspy.clients.fdsn import Client
from obspy import UTCDateTime

In [16]:
def fetch_data(start_time, end_time, min_magnitude):
    # Connect to IRIS
    t1 = UTCDateTime(start_time)
    t2 = UTCDateTime(end_time)
    client = Client("IRIS")
    catalog = client.get_events(starttime=t1, endtime=t2, minmagnitude=min_magnitude)

    # Prepare data
    times, magnitudes, depths, longitudes, latitudes = [], [], [], [], []
    for event in catalog:
        origin = event.origins[0]
        times.append(origin.time.datetime)
        magnitudes.append(event.magnitudes[0].mag)
        depths.append(origin.depth / 1000)  # Depth in km
        longitudes.append(origin.longitude)
        latitudes.append(origin.latitude)

    df = pd.DataFrame({
        'time': times,
        'magnitude': magnitudes,
        'depth': depths,
        'longitude': longitudes,
        'latitude': latitudes
    })

    df['time_bin'] = df['time'].dt.to_period('M').dt.to_timestamp()
    grouped = df.groupby('time_bin').size().reset_index(name='count')
    return df, grouped

In [17]:
# Step 1: Fetch data
def fetch_data(start_time, end_time, min_magnitude):
    # Connect to IRIS
    t1 = UTCDateTime(start_time)
    t2 = UTCDateTime(end_time)
    client = Client("IRIS")
    catalog = client.get_events(starttime=t1, endtime=t2, minmagnitude=min_magnitude)

    # Prepare data
    times, magnitudes, depths, longitudes, latitudes = [], [], [], [], []
    for event in catalog:
        origin = event.origins[0]
        times.append(origin.time.datetime)
        magnitudes.append(event.magnitudes[0].mag)
        depths.append(origin.depth / 1000)  # Depth in km
        longitudes.append(origin.longitude)
        latitudes.append(origin.latitude)

    df = pd.DataFrame({
        'time': times,
        'magnitude': magnitudes,
        'depth': depths,
        'longitude': longitudes,
        'latitude': latitudes
    })

    df['time_bin'] = df['time'].dt.to_period('M').dt.to_timestamp()
    grouped = df.groupby('time_bin').size().reset_index(name='count')
    return df, grouped

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout with additional controls
app.layout = html.Div([
    html.H1("Interactive Earthquake Visualization", style={'textAlign': 'center'}),
    
    # Year Range and Magnitude Input
    html.Div([
        html.Label("Select Year Range:", style={'marginLeft': '20px'}),
        dcc.RangeSlider(
            id="year-range-slider",
            min=2000, max=2023, step=1,
            marks={year: str(year) for year in range(2000, 2024, 5)},
            value=[2010, 2015]
        ),
        html.Label("Minimum Magnitude: ", style={'marginLeft': '20px'}),
        dcc.Input(id="min-magnitude-input", type="number", value=7.5, step=0.1),
        html.Div(id="error-message", style={'color': 'red', 'marginTop': '10px'}),
    ], style={'marginBottom': '20px'}),
    
    # Graphs
    dcc.Graph(id="scatter-plot"),
    dcc.Graph(id="bar-chart")
])

@app.callback(
    [Output("scatter-plot", "figure"),
     Output("bar-chart", "figure"),
     Output("error-message", "children")],
    [Input("year-range-slider", "value"),
     Input("min-magnitude-input", "value"),
     Input("bar-chart", "clickData"),
     Input("scatter-plot", "clickData")]
)
def update_data(year_range, min_magnitude, bar_click, scatter_click):
    start_year, end_year = year_range
    start_time = f"{start_year}-01-01"
    end_time = f"{end_year}-12-31"

    # Initialize empty highlight condition
    highlight_condition = None

    try:
        # Fetch data
        t1 = UTCDateTime(start_time)
        t2 = UTCDateTime(end_time)
        client = Client("IRIS")
        catalog = client.get_events(starttime=t1, endtime=t2, minmagnitude=min_magnitude)

        # Prepare data
        times, magnitudes, depths, longitudes, latitudes = [], [], [], [], []
        for event in catalog:
            origin = event.origins[0]
            times.append(origin.time.datetime)
            magnitudes.append(event.magnitudes[0].mag)
            depths.append(origin.depth / 1000)  # Depth in km
            longitudes.append(origin.longitude)
            latitudes.append(origin.latitude)

        df = pd.DataFrame({
            'time': times,
            'magnitude': magnitudes,
            'depth': depths,
            'longitude': longitudes,
            'latitude': latitudes
        })

        # Check if data is too large
        if len(df) > 10000:
            return {}, {}, "Data size too large. Please reduce the range or increase the minimum magnitude."

        # Group data by time_bin for bar chart
        df['time_bin'] = df['time'].dt.to_period('M').dt.to_timestamp()
        grouped = df.groupby('time_bin').size().reset_index(name='count')

        # Determine the triggered input
        triggered_id = ctx.triggered_id

        if triggered_id == "bar-chart" and bar_click:
            click_date = bar_click["points"][0]["x"]
            highlight_condition = df['time_bin'] == pd.Timestamp(click_date)

        elif triggered_id == "scatter-plot" and scatter_click:
            click_lon = scatter_click["points"][0]["lon"]
            click_lat = scatter_click["points"][0]["lat"]
            tolerance = 0.01
            selected = df[
                (df['longitude'].between(click_lon - tolerance, click_lon + tolerance)) &
                (df['latitude'].between(click_lat - tolerance, click_lat + tolerance))
            ]
            if not selected.empty:
                click_date = selected.iloc[0]['time_bin']
                highlight_condition = df['time_bin'] == click_date

        # Scatter Plot (Map)
        scatter_fig = go.Figure(go.Scattergeo(
            lon=df['longitude'],
            lat=df['latitude'],
            text=[
                f"Mag: {mag}<br>Depth: {depth} km<br>Time: {time}"
                for mag, depth, time in zip(df['magnitude'], df['depth'], df['time'])
            ],
            marker=dict(
                size=df['magnitude'] * 2,
                color=df['depth'],
                colorscale='Viridis_r',
                showscale=True,
                colorbar=dict(title="Depth (km)")
            ),
            mode='markers',
            name="Earthquakes"
        ))
        scatter_fig.update_layout(
            geo=dict(
                resolution=50,
                showland=True,
                landcolor="rgb(243, 243, 243)",
                showcountries=True,
                countrycolor="rgb(204, 204, 204)"
            ),
            margin=dict(l=50, r=50, t=50, b=50),
            title="Earthquake Map"
        )

        # Highlight selected earthquakes on the map
        if highlight_condition is not None:
            highlighted = df[highlight_condition]
            scatter_fig.add_trace(go.Scattergeo(
                lon=highlighted['longitude'],
                lat=highlighted['latitude'],
                mode='markers',
                marker=dict(size=12, color='red', opacity=0.8),
                name="Highlighted"
            ))

        # Bar Chart
        bar_fig = go.Figure(go.Bar(
            x=grouped['time_bin'],
            y=grouped['count'],
            marker=dict(color="skyblue"),
            name="Earthquakes per Month"
        ))
        bar_fig.update_layout(
            title="Number of Earthquakes Over Time",
            xaxis=dict(title="Time"),
            yaxis=dict(title="Number of Earthquakes"),
        )

        # Highlight selected bar
        if highlight_condition is not None:
            highlighted_time_bins = grouped[grouped['time_bin'].isin(highlighted['time_bin'])]
            bar_fig.add_trace(go.Bar(
                x=highlighted_time_bins['time_bin'],
                y=highlighted_time_bins['count'],
                marker=dict(color='red', opacity=0.8),
                name="Highlighted"
            ))

        return scatter_fig, bar_fig, ""

    except Exception as e:
        # Handle errors gracefully
        return {}, {}, f"Error fetching data: {str(e)}"

In [18]:
app.run_server(debug=True, jupyter_mode = 'external')

Dash app running on http://127.0.0.1:8050/
